<a href="https://colab.research.google.com/github/MohmedZeibreal/NLP-Project-3-/blob/main/NLP_Project_3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# NLP

In [ ]:
# ======================================================
# CELL 1 — TRAINING TABLE + MODEL TRAINER + DELETE OPTION
# ======================================================

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# -------------------------------------------
# 1. TRAINING TABLE (User Question + Bot Answer)
# -------------------------------------------
df = pd.DataFrame({
    "User Question": [""],
    "Bot Answer": [""]
})

train_table = widgets.Output()

with train_table:
    display(df)

# Editable textboxes
user_box = widgets.Text(description="User:")
bot_box = widgets.Text(description="Bot:")

add_btn = widgets.Button(description="Add Pair", button_style="success")

def update_dropdown():
    """Refresh delete dropdown list."""
    delete_options = [
        f"{i}: {df.loc[i, 'User Question']} → {df.loc[i, 'Bot Answer']}"
        for i in df.index
    ]
    delete_dropdown.options = delete_options

def add_pair(_):
    global df
    if user_box.value.strip() and bot_box.value.strip():
        df.loc[len(df)] = [user_box.value, bot_box.value]

        # Update displayed table
        with train_table:
            clear_output()
            display(df)

        # Reset inputs
        user_box.value = ""
        bot_box.value = ""

        # Update delete list
        update_dropdown()

add_btn.on_click(add_pair)

# -------------------------------------------
# DELETE OPTION
# -------------------------------------------
delete_dropdown = widgets.Dropdown(description="Delete Row:", options=[])
delete_btn = widgets.Button(description="Delete Selected", button_style="danger")

def delete_pair(_):
    global df

    if not delete_dropdown.options:
        return

    # Get selected index
    selected_text = delete_dropdown.value
    idx = int(selected_text.split(":")[0])

    # Remove row
    df = df.drop(index=idx).reset_index(drop=True)

    # Update display
    with train_table:
        clear_output()
        display(df)

    # Refresh dropdown
    update_dropdown()

delete_btn.on_click(delete_pair)

# -------------------------------------------
# 2. TRAIN MODEL BUTTON
# -------------------------------------------
train_btn = widgets.Button(description="Train Model", button_style="primary")
train_output = widgets.Output()

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

def train_model(_):
    global model, vectorizer

    with train_output:
        clear_output()
        print("Training model...")

    X = df["User Question"].astype(str).values
    y = df["Bot Answer"].astype(str).values

    vectorizer = TfidfVectorizer()
    X_vect = vectorizer.fit_transform(X)

    model = LogisticRegression()
    model.fit(X_vect, y)

    with train_output:
        print("Model trained successfully!")

def chatbot_response(message):
    X_vect = vectorizer.transform([message])
    return model.predict(X_vect)[0]

train_btn.on_click(train_model)

# -------------------------------------------
# DISPLAY UI
# -------------------------------------------
display(
    widgets.VBox([
        widgets.HTML("<h2>Training Table</h2>"),
        user_box, bot_box, add_btn,
        train_table,
        widgets.HTML("<h3>Delete a Question/Answer Pair</h3>"),
        delete_dropdown, delete_btn,
        widgets.HTML("<hr><h2>Train the Model</h2>"),
        train_btn, train_output
    ])
)

# Initialize dropdown on load
update_dropdown()


In [ ]:
# ======================================================
# CELL 2 — AI CHATBOT WEBSITE (GRADIO UI)
# ======================================================

import gradio as gr

# Uses chatbot_response() from Cell 1

with gr.Blocks(title="ChatBox") as demo:
    gr.Markdown("<h1 style='text-align:center;'>ChatBox</h1>")
    gr.Markdown("<p style='text-align:center;'>Chat With chatbox</p>")

    chatbox = gr.Chatbot(height=450)
    msg = gr.Textbox(placeholder="Type ur doubts...", label="Clear ur Doubts")
    send_btn = gr.Button("Send", variant="primary")

    def chat_fn(user_message, history):
        bot_reply = chatbot_response(user_message)
        history = history + [(user_message, bot_reply)]
        return history, ""

    send_btn.click(chat_fn, [msg, chatbox], [chatbox, msg])

demo.launch()


/tmp/ipython-input-3286292991.py:13: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbox = gr.Chatbot(height=450)
/tmp/ipython-input-3286292991.py:13: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbox = gr.Chatbot(height=450)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ccebe9cf1d62e77e3f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
